#Set up


In [1]:
# Import libraries
from datetime import timedelta
import os

import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

from IPython.display import display, HTML, Image
%matplotlib inline

plt.style.use('ggplot')
plt.rcParams.update({'font.size': 20})

# Access data using Google BigQuery.
from google.colab import auth
from google.cloud import bigquery

# authenticate
auth.authenticate_user()

# Set up environment variables
##project_id = 'stoked-sanctum-433501-f8'
project_id='mimic-432400'
if project_id == 'CHANGE-ME':
  raise ValueError('You must change project_id to your GCP project.')
os.environ["GOOGLE_CLOUD_PROJECT"] = project_id

# Read data from BigQuery into pandas dataframes.
def run_query(query, project_id=project_id):
  return pd.io.gbq.read_gbq(
      query,
      project_id=project_id,
      dialect='standard')

# set the dataset
# if you want to use the demo, change this to mimic_demo
dataset = 'mimiciv'

# Subject selection

In [2]:
epinephrine_pharmacy=run_query("""SELECT Subject_id,hadm_id,pharmacy_id,poe_id,starttime,stoptime, medication, proc_type,status,route,frequency,disp_sched,doses_per_24_hrs,duration,duration_interval
  FROM physionet-data.mimic_hosp.pharmacy
  WHERE Medication LIKE '%Epinephrine%'""")

epinephrine_input=run_query("""SELECT subject_id,hadm_id,starttime,endtime,amount,amountuom,label
 FROM physionet-data.mimic_icu.inputevents ie
  LEFT JOIN physionet-data.mimic_icu.d_items i
    ON i.itemid=ie.itemid
  WHERE linksto='inputevents' and category='Medications'
    AND label IN ('Epinephrine', 'Epinephrine.')""")

epinephrine=run_query("""SELECT
    subject_id, e.stay_id, hadm_id,linkorderid,vaso_rate,vaso_amount,starttime,endtime
  FROM physionet-data.mimic_derived.epinephrine e
  JOIN physionet-data.mimic_derived.icustay_detail s
    ON e.stay_id=s.stay_id""")##may be a duplicate

diagnosis_filter = run_query("""SELECT * FROM `physionet-data.mimic_hosp.diagnoses_icd`
  WHERE icd_version=10
  AND (icd_code LIKE 'T780%' OR icd_code LIKE 'T782%'
    OR icd_code LIKE 'T886%' OR icd_code LIKE 'T805%')""")


<ipython-input-1-21d6a970b20c>:33: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  return pd.io.gbq.read_gbq(


In [3]:
def extract_day(df, column):

    df[column] = pd.to_datetime(df[column])

    df['day'] = df[column].dt.date  # Extract only the day

    return df[['hadm_id', 'day']]


In [4]:
# merging toepinephrine queries to get a single list of hadms_ids
epinephrine_pharmacy_keys=extract_day(epinephrine_pharmacy,'starttime')
epinephrine_input_keys=extract_day(epinephrine_input,'starttime')
epinephrine_keys=extract_day(epinephrine,'starttime')



keys = pd.concat([epinephrine_pharmacy_keys,epinephrine_input_keys,epinephrine_keys,diagnosis_filter])

In [5]:
hadm_ids_list = keys['hadm_id'].unique()

# Get data to use in predictions

In [6]:
# WHERE hadm_id IN ({','.join([str(hadm_id) for hadm_id in hadm_ids_list])})

query = f"""SELECT
    hadm_id,v.stay_id, v.subject_id,charttime as charttime_vital,heart_rate,sbp,dbp,mbp,sbp_ni,dbp_ni,mbp_ni,resp_rate,spo2
  FROM physionet-data.mimic_derived.vitalsign v
  LEFT JOIN physionet-data.mimic_derived.icustay_detail s
    ON v.stay_id=s.stay_id
    WHERE hadm_id IN ({','.join([str(hadm_id) for hadm_id in hadm_ids_list])})"""

vitalsign=run_query(query)
vitalsign.head()

<ipython-input-1-21d6a970b20c>:33: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  return pd.io.gbq.read_gbq(


,hadm_id,stay_id,subject_id,charttime_vital,heart_rate,sbp,dbp,mbp,sbp_ni,dbp_ni,mbp_ni,resp_rate,spo2
0,27332538,37329406,18908516,2180-05-24 11:00:00,98.0,151.0,86.0,105.0,NaN,NaN,NaN,27.5,100.0
1,20969955,33576993,19624089,2163-12-15 16:00:00,74.0,85.0,28.0,41.0,NaN,NaN,NaN,27.5,94.0
2,20482954,33460567,13777050,2144-10-27 15:00:00,81.0,51.0,26.0,31.0,NaN,NaN,NaN,19.5,NaN
3,27694894,39683743,19822462,2162-08-21 23:00:00,84.0,106.0,37.0,47.5,NaN,NaN,NaN,26.5,93.0
4,25361762,30063714,11570162,2133-01-12 12:00:00,121.0,96.5,53.5,65.5,108.0,54.0,67.0,36.0,96.0


In [7]:

# saving to csv to avoid having to rerun query, takes too long
vitalsign.to_csv('vitalsign.csv')

In [8]:
# if loading from csv
vitalsign=pd.read_csv('vitalsign.csv')

In [9]:
#convert starttdate on vitalsign to day and only keep the DAY
vitalsign['charttime_vital'] = pd.to_datetime(vitalsign['charttime_vital'])
vitalsign['day'] = vitalsign['charttime_vital'].dt.date

In [10]:
vitalsign.head()

,Unnamed: 0,hadm_id,stay_id,subject_id,charttime_vital,heart_rate,sbp,dbp,mbp,sbp_ni,dbp_ni,mbp_ni,resp_rate,spo2,day
0,0,27332538,37329406,18908516,2180-05-24 11:00:00,98.0,151.0,86.0,105.0,NaN,NaN,NaN,27.5,100.0,2180-05-24
1,1,20969955,33576993,19624089,2163-12-15 16:00:00,74.0,85.0,28.0,41.0,NaN,NaN,NaN,27.5,94.0,2163-12-15
2,2,20482954,33460567,13777050,2144-10-27 15:00:00,81.0,51.0,26.0,31.0,NaN,NaN,NaN,19.5,NaN,2144-10-27
3,3,27694894,39683743,19822462,2162-08-21 23:00:00,84.0,106.0,37.0,47.5,NaN,NaN,NaN,26.5,93.0,2162-08-21
4,4,25361762,30063714,11570162,2133-01-12 12:00:00,121.0,96.5,53.5,65.5,108.0,54.0,67.0,36.0,96.0,2133-01-12


In [11]:
# next we want to aggregate vitalsign per day and per hadm_id

agg_vital_sign = vitalsign.groupby(['hadm_id', 'day']).agg(
    heart_rate=('heart_rate', 'mean'),
    sbp=('sbp', 'mean'),
    dbp=('dbp', 'mean'),
    mbp=('mbp', 'mean'),
    sbp_ni=('sbp_ni', 'mean'),
    dbp_ni=('dbp_ni', 'mean'),
    mbp_ni=('mbp_ni', 'mean'),
    resp_rate=('resp_rate', 'mean'),
    spo2=('spo2', 'mean')
).reset_index()
agg_vital_sign.head()

,hadm_id,day,heart_rate,sbp,dbp,mbp,sbp_ni,dbp_ni,mbp_ni,resp_rate,spo2
0,20002267,2114-01-15,84.937500,98.800000,69.775000,79.175000,95.428571,75.428571,83.000000,15.305556,100.000000
1,20002267,2114-01-16,95.961538,111.032258,75.919355,86.951613,108.733333,83.333333,92.266667,20.206897,99.034483
2,20002267,2114-01-17,77.958333,112.480000,80.760000,91.560000,110.187500,84.937500,94.187500,19.083333,97.500000
3,20002267,2114-01-18,77.666667,117.222222,85.666667,96.000000,117.222222,85.666667,96.000000,23.888889,95.666667
4,20003587,2146-04-17,92.395833,104.187500,52.406250,67.187500,96.666667,43.333333,56.166667,31.010204,89.641026


In [12]:
#we repeat for chemistry
query = f"""SELECT
  subject_id, hadm_id, charttime, specimen_id, creatinine
  FROM physionet-data.mimic_derived.chemistry
  WHERE hadm_id IN ({','.join([str(hadm_id) for hadm_id in hadm_ids_list])})"""

chemistry=run_query(query)
chemistry.head(10)


<ipython-input-1-21d6a970b20c>:33: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  return pd.io.gbq.read_gbq(


,subject_id,hadm_id,charttime,specimen_id,creatinine
0,15394383,29563142,2136-05-10 07:15:00,83015920,3.5
1,11492213,22184277,2142-01-16 14:03:00,22895428,7.4
2,18891052,23399964,2111-06-17 08:40:00,85931541,4.1
3,11943854,23404999,2135-09-18 10:23:00,40455056,5.5
4,15114658,26406456,2120-01-26 00:00:00,44884407,3.8
5,12251785,22844070,2176-01-03 03:29:00,49361889,4.1
6,18432749,25544888,2182-09-25 05:42:00,97141154,3.0
7,11511428,21044939,2141-02-02 21:53:00,19672525,3.4
8,10685870,24659736,2166-04-29 05:56:00,97863480,3.9
9,14786549,27568834,2186-06-25 01:59:00,11467842,9.6


In [13]:
# for skin, we just get the warn column and the charttime as day

query = f"""SELECT subject_id,hadm_id,charttime as charttime_skin,storetime,value,warning,label
 FROM physionet-data.mimic_icu.chartevents ie
  LEFT JOIN physionet-data.mimic_icu.d_items i
    ON i.itemid=ie.itemid
  WHERE linksto='chartevents' and Category='Skin - Assessment'
  AND label IN ('Skin Condition','Skin Integrity','Skin Color', 'Skin Temperature')
  AND hadm_id IN ({','.join([str(hadm_id) for hadm_id in hadm_ids_list])})"""

skin_chart = run_query(query)
skin_chart.head()

<ipython-input-1-21d6a970b20c>:33: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  return pd.io.gbq.read_gbq(


,subject_id,hadm_id,charttime_skin,storetime,value,warning,label
0,17536222,26728411,2164-01-26 16:00:00,2164-01-26 16:20:00,Ashen,0,Skin Color
1,19326831,29981653,2157-09-02 12:56:00,2157-09-02 12:56:00,Flushed,0,Skin Color
2,16824843,27776968,2167-02-18 00:00:00,2167-02-18 04:34:00,Flushed,0,Skin Color
3,15850072,24051036,2141-01-27 08:05:00,2141-01-27 12:23:00,Clammy,0,Skin Condition
4,19869901,20859990,2188-01-30 08:00:00,2188-01-30 08:51:00,Hot,0,Skin Temperature


In [14]:
# we exctarct warning, and convert the charttime to day

skin_chart['day'] = pd.to_datetime(skin_chart['charttime_skin'])
skin_chart['day'] = skin_chart['charttime_skin'].dt.date

In [15]:
skin_chart.to_csv('skin_chart.csv')

In [16]:
# we aggregate skin by day and hadm_id. We only care about the "warning" column.
# if a day has any warning equal to 1, we set it to 1 for that day

skin_agg = skin_chart.groupby(['hadm_id', 'day']).agg(
    warning_skin=('warning', 'max')
).reset_index()
skin_agg.head()

,hadm_id,day,warning_skin
0,20002267,2114-01-15,0
1,20002267,2114-01-16,0
2,20002267,2114-01-17,0
3,20002267,2114-01-18,0
4,20003587,2146-04-17,0


In [17]:
# we repeat again for gatrintestinal

query = f"""SELECT subject_id,hadm_id,charttime as charttime_gastro,storetime,value,warning,label
 FROM physionet-data.mimic_icu.chartevents ie
  LEFT JOIN physionet-data.mimic_icu.d_items i
    ON i.itemid=ie.itemid
  WHERE linksto='chartevents' and Category='Toxicology'
  AND label IN ('Nausea and Vomiting (CIWA)','GI Upset (COWS)','Nausea and Vomiting (CINA)')
  AND value NOT IN ("No N/V", "No nausea, no vomiting","No GI symptoms","No nausea, no vomiting.")
  AND hadm_id IN ({','.join([str(hadm_id) for hadm_id in hadm_ids_list])})"""
gastrointensintal_chart=run_query(query)

<ipython-input-1-21d6a970b20c>:33: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  return pd.io.gbq.read_gbq(


In [18]:
gastrointensintal_chart.head()

,subject_id,hadm_id,charttime_gastro,storetime,value,warning,label
0,18481232,22380655,2188-08-05 08:00:00,2188-08-05 10:31:00,"Intense Nausea, Dry Heaves",0,Nausea and Vomiting (CIWA)
1,11949990,28691803,2119-10-10 02:27:00,2119-10-10 02:28:00,"Mild Nausea, No Vomiting",0,Nausea and Vomiting (CIWA)
2,12653086,20373749,2166-07-06 00:20:00,2166-07-06 00:20:00,"Mild Nausea, No Vomiting...",0,Nausea and Vomiting (CIWA)
3,11070185,24518501,2122-12-23 02:00:00,2122-12-23 02:15:00,"Mild Nausea, No Vomiting",0,Nausea and Vomiting (CIWA)
4,16333512,21773493,2183-01-08 16:00:00,2183-01-08 18:26:00,"Intense Nausea, Dry Heaves",0,Nausea and Vomiting (CIWA)


In [19]:
gastrointensintal_chart['day'] = pd.to_datetime(gastrointensintal_chart['charttime_gastro'])
gastrointensintal_chart['day'] = gastrointensintal_chart['charttime_gastro'].dt.date

gastrointensintal_chart.head()

,subject_id,hadm_id,charttime_gastro,storetime,value,warning,label,day
0,18481232,22380655,2188-08-05 08:00:00,2188-08-05 10:31:00,"Intense Nausea, Dry Heaves",0,Nausea and Vomiting (CIWA),2188-08-05
1,11949990,28691803,2119-10-10 02:27:00,2119-10-10 02:28:00,"Mild Nausea, No Vomiting",0,Nausea and Vomiting (CIWA),2119-10-10
2,12653086,20373749,2166-07-06 00:20:00,2166-07-06 00:20:00,"Mild Nausea, No Vomiting...",0,Nausea and Vomiting (CIWA),2166-07-06
3,11070185,24518501,2122-12-23 02:00:00,2122-12-23 02:15:00,"Mild Nausea, No Vomiting",0,Nausea and Vomiting (CIWA),2122-12-23
4,16333512,21773493,2183-01-08 16:00:00,2183-01-08 18:26:00,"Intense Nausea, Dry Heaves",0,Nausea and Vomiting (CIWA),2183-01-08


In [20]:
gastrointensintal_chart.to_csv('gastrointensintal_chart.csv')

In [21]:
gastro_agg = gastrointensintal_chart.groupby(['hadm_id', 'day']).agg(
    warning_gastro=('warning', 'max')
).reset_index()
gastro_agg.head()

,hadm_id,day,warning_gastro
0,20373749,2166-07-05,0
1,20373749,2166-07-06,0
2,20883139,2131-09-20,0
3,20966809,2115-05-18,0
4,20966809,2115-05-19,0


In [ ]:
#finally, we repeat for blood differential


In [22]:
query = f"""SELECT
  subject_id, hadm_id, charttime, specimen_id, eosinophils_abs, eosinophils
  FROM physionet-data.mimic_derived.blood_differential
  WHERE hadm_id IN ({','.join([str(hadm_id) for hadm_id in hadm_ids_list])})"""

blood_differential=run_query(query)

<ipython-input-1-21d6a970b20c>:33: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  return pd.io.gbq.read_gbq(


In [23]:
blood_differential.head()

,subject_id,hadm_id,charttime,specimen_id,eosinophils_abs,eosinophils
0,13702880,21532971,2189-11-04 00:09:00,73596400,0.00,0.0
1,16742822,28329092,2150-02-11 00:00:00,588599,0.01,1.0
2,13830142,25898548,2147-12-03 00:00:00,55170479,0.00,0.0
3,14464333,24356178,2116-12-10 01:05:00,72830818,123.20,4.0
4,15751968,20963960,2128-04-05 00:15:00,84472607,0.00,0.0


In [24]:
blood_differential['day'] = pd.to_datetime(blood_differential['charttime'])
blood_differential['day'] = blood_differential['charttime'].dt.date


In [25]:
blood_differential.to_csv('blood_differential.csv')

In [26]:

blood_agg = blood_differential.groupby(['hadm_id', 'day']).agg(
    eosinophils_abs=('eosinophils_abs', 'mean'),
    eosinophils=('eosinophils', 'mean')
).reset_index()
blood_agg.head()

,hadm_id,day,eosinophils_abs,eosinophils
0,20002267,2114-01-15,0.06,1.2
1,20002267,2114-01-16,NaN,NaN
2,20002267,2114-01-17,NaN,NaN
3,20002267,2114-01-18,NaN,NaN
4,20002267,2114-01-19,NaN,NaN


In [27]:
# we now merge all, using day and hadm_id as our primary key

merged_df = pd.merge(agg_vital_sign, skin_agg, on=['hadm_id', 'day'], how='outer')
merged_df = pd.merge(merged_df, gastro_agg, on=['hadm_id', 'day'], how='outer')
merged_df = pd.merge(merged_df, blood_agg, on=['hadm_id', 'day'], how='outer')
merged_df.head()

,hadm_id,day,heart_rate,sbp,dbp,mbp,sbp_ni,dbp_ni,mbp_ni,resp_rate,spo2,warning_skin,warning_gastro,eosinophils_abs,eosinophils
0,20002267,2114-01-15,84.937500,98.800000,69.775000,79.175000,95.428571,75.428571,83.000000,15.305556,100.000000,0,<NA>,0.06,1.2
1,20002267,2114-01-16,95.961538,111.032258,75.919355,86.951613,108.733333,83.333333,92.266667,20.206897,99.034483,0,<NA>,NaN,NaN
2,20002267,2114-01-17,77.958333,112.480000,80.760000,91.560000,110.187500,84.937500,94.187500,19.083333,97.500000,0,<NA>,NaN,NaN
3,20002267,2114-01-18,77.666667,117.222222,85.666667,96.000000,117.222222,85.666667,96.000000,23.888889,95.666667,0,<NA>,NaN,NaN
4,20002267,2114-01-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,NaN,NaN


# Target column creation

In [28]:
# finally, we add our target column, a boolean column off the hadm_id having a
# diagnosis AND for the day being administered epinophrine

query = f"""SELECT * FROM `physionet-data.mimic_hosp.diagnoses_icd`
  WHERE icd_version=10
  AND (icd_code LIKE 'T780%' OR icd_code LIKE 'T782%'
    OR icd_code LIKE 'T886%' OR icd_code LIKE 'T805%')
  AND hadm_id IN ({','.join([str(hadm_id) for hadm_id in hadm_ids_list])})"""

diagnosis = run_query(query)
diagnosis.head()

<ipython-input-1-21d6a970b20c>:33: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  return pd.io.gbq.read_gbq(


,subject_id,hadm_id,seq_num,icd_code,icd_version
0,17743440,20320809,1,T886XXA,10
1,13151205,24727419,1,T7809XA,10
2,17600374,28695895,1,T886XXA,10
3,18109577,24540659,1,T886XXA,10
4,10863078,26102500,1,T782XXA,10


In [29]:
# now we add our target column. If the hadm_id is both in diagnosis and our merged
# hadm_list, then we set it to true. If it is missing from either it is false.
# If it was not present

merged_df['target'] = merged_df['hadm_id'].isin(diagnosis['hadm_id'])
merged_df.head()

,hadm_id,day,heart_rate,sbp,dbp,mbp,sbp_ni,dbp_ni,mbp_ni,resp_rate,spo2,warning_skin,warning_gastro,eosinophils_abs,eosinophils,target
0,20002267,2114-01-15,84.937500,98.800000,69.775000,79.175000,95.428571,75.428571,83.000000,15.305556,100.000000,0,<NA>,0.06,1.2,False
1,20002267,2114-01-16,95.961538,111.032258,75.919355,86.951613,108.733333,83.333333,92.266667,20.206897,99.034483,0,<NA>,NaN,NaN,False
2,20002267,2114-01-17,77.958333,112.480000,80.760000,91.560000,110.187500,84.937500,94.187500,19.083333,97.500000,0,<NA>,NaN,NaN,False
3,20002267,2114-01-18,77.666667,117.222222,85.666667,96.000000,117.222222,85.666667,96.000000,23.888889,95.666667,0,<NA>,NaN,NaN,False
4,20002267,2114-01-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,NaN,NaN,False


In [30]:
# we visualize our distribution of target

merged_df['target'].value_counts()

,count
target,
False,81341
True,914


In [34]:
import pandas as pd
from sklearn.impute import SimpleImputer

merged_df = merged_df.replace({pd.NA: np.nan})
#replace empty values in warning_skin and warning_gastro with false
merged_df['warning_skin'] = merged_df['warning_skin'].fillna(0)
merged_df['warning_gastro'] = merged_df['warning_gastro'].fillna(0)
# Create a SimpleImputer to handle missing values
# Strategy can be 'mean', 'median', 'most_frequent', or 'constant'
imputer = SimpleImputer(strategy='most_frequent')

# Apply the imputer to the feature columns of your DataFrame
X = merged_df.drop('target', axis=1)  # Features
X_imputed = imputer.fit_transform(X) # fit and transform the data

# Create a new DataFrame with imputed values
X_imputed_df = pd.DataFrame(X_imputed, columns=X.columns, index=X.index) #new dataframe with original column names and index

# Get the target values
y = merged_df['target']

X_imputed_df['day'] = pd.to_datetime(X_imputed_df['day']).apply(lambda x: x.toordinal())
merged2_df = pd.concat([X_imputed_df, y], axis=1)


In [35]:
merged2_df.to_csv('merged_df.csv')

In [ ]:
# Due to tha class imbalance, we apply SMOTE to generate synthetic data for it
# be practical to use in ML models target=true has the least count

import pandas as pd
from imblearn.over_sampling import SMOTE
from sklearn.impute import SimpleImputer

merged_df = merged_df.replace({pd.NA: np.nan})
#replace empty values in warning_skin and warning_gastro with false
merged_df['warning_skin'] = merged_df['warning_skin'].fillna(0)
merged_df['warning_gastro'] = merged_df['warning_gastro'].fillna(0)
# Create a SimpleImputer to handle missing values
# Strategy can be 'mean', 'median', 'most_frequent', or 'constant'
imputer = SimpleImputer(strategy='most_frequent')

# Apply the imputer to the feature columns of your DataFrame
X = merged_df.drop('target', axis=1)  # Features
X_imputed = imputer.fit_transform(X) # fit and transform the data

# Create a new DataFrame with imputed values
X_imputed_df = pd.DataFrame(X_imputed, columns=X.columns, index=X.index) #new dataframe with original column names and index

# Get the target values
y = merged_df['target']

X_imputed_df['day'] = pd.to_datetime(X_imputed_df['day']).apply(lambda x: x.toordinal())

# Now apply SMOTE to the imputed data
smote = SMOTE(sampling_strategy='minority')
X_resampled, y_resampled = smote.fit_resample(X_imputed_df, y)

smote_df = pd.concat([X_resampled, y_resampled], axis=1)
smote_df.head()

smote_df['target'].value_counts()

,count
target,
False,321810
True,321810


In [ ]:
smote_df.to_csv('smote_df.csv')